### Save and Retrive Data from Redis
- https://redis.com/

In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
# Connect to Redis Client
# redis-18692.c57.us-east-1-4.ec2.cloud.redislabs.com:18692
# wCneC8tYJRuw0ovU9Kacbr9kgY4UmQdb
hostname = 'redis-14249.c239.us-east-1-2.ec2.cloud.redislabs.com'
portnumber = 14249
password = '5ASm8yEK2Bu8o4aqqXzkM1otOfhyM57z'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)


In [3]:
r.ping()

True

In [4]:
#r.keys()

#### Load the data (numpy zip)

In [5]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,student,"[0.43222713, 1.074822, -0.12857957, -1.8332443..."
1,Barack_Obama,teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ..."
2,Chris_Evans,student,"[0.72012275, -1.4683539, -0.77757514, -1.46026..."
3,Morgan_Freeman,teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711..."
4,Scarlett Johansson,student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578..."


In [6]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,student,"[0.43222713, 1.074822, -0.12857957, -1.8332443...",Angelina Jolie@student
1,Barack_Obama,teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ...",Barack_Obama@teacher
2,Chris_Evans,student,"[0.72012275, -1.4683539, -0.77757514, -1.46026...",Chris_Evans@student
3,Morgan_Freeman,teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711...",Morgan_Freeman@teacher
4,Scarlett Johansson,student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578...",Scarlett Johansson@student


In [7]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

## Save data in Redis
- redis hashes
- keyname = 'academy:register'

In [8]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis cloud
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [9]:
vector.dtype

dtype('float32')

### Retrive Records from Redis

In [10]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Abhijeeth@Student'                     b'\xb7\x8c\xe6\xbc\x10XV\xbb\xf6\xf5"\xbd.\x04...
b'kohli@Student'                         b'\xc8\x07\x8e\xbfl\x88\xbe>2E\xf9=\x94\xa9.\x...
b'abhigna@Student'                       b'\x0f/\x00\xc0\x14\xae\x02\xbf\x1bm7?\x8c\x93...
b'Lionel Messi@Student'                  b'w\r-\xc0$\x82\xb3\xbfuz\xca>A\x85.?)\xa5\x9e...
b'Morgan_Freeman@teacher'                b'\xfa\xb9\xf8>\x8e\x8b\x1e\xbf\xeb\x88\xd0\xb...
b'Barack_Obama@teacher'                  b'\xa5\xd7r?\xf6\xef\x0e\xbf\x96?\xc2=\x0f\xcb...
b'Nihitha Vadlamuri@Student'             b'w\x90\xcf>f\xf3n\xbf\xf3\xc5\x1b? \xe1r\xbf6...
b'Scarlett Johansson@student'            b'\xc6\xe5C>\xfd8\xfa\xbe\xb3_\xd8\xbf\xd6\x89...
b'ronaldo@Student_21WU0102047'           b'L\x8a\xdc?Z\xda1\xc0\xbc\x1e\x12><\xb7\xde\x...
b'Chris_Evans@student'                   b'\xf7Y8?\x05\xf3\xbb\xbf*\x0fG\xbf\x13\xea\xb...
b'Angelina Jolie@student'                b'\xe0L\xdd>\xc4\x93\x89?]\xaa\x03\xbe\xc0\xa7...

In [11]:
#np.frombuffer(retrive_series[b'Morgan Freeman@Teacher'],dtype=np.float32).shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))

In [12]:
retrive_series.index = index

In [13]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Abhijeeth@Student,"[-0.02814327, -0.0032706298, -0.039785348, -0...."
1,kohli@Student,"[-1.1096125, 0.37213457, 0.12171401, -0.682275..."
2,abhigna@Student,"[-2.0028722, -0.5104687, 0.71650857, -0.830376..."
3,Lionel Messi@Student,"[-2.7039468, -1.4024091, 0.39546552, 0.6817208..."
4,Morgan_Freeman@teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711..."
5,Barack_Obama@teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ..."
6,Nihitha Vadlamuri@Student,"[0.40539905, -0.93340147, 0.6084892, -0.948747..."
7,Scarlett Johansson@student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578..."
8,ronaldo@Student_21WU0102047,"[1.7229705, -2.7789521, 0.14269537, -1.7399669..."
9,Chris_Evans@student,"[0.72012275, -1.4683539, -0.77757514, -1.46026..."


In [14]:
print(retrive_df.columns)



Index(['name_role', 'facial_features'], dtype='object')


In [15]:
name_to_delete = 'Abhigna Ragala@Student_21WU0102051'
retrive_df = retrive_df[~retrive_df['name_role'].str.contains(name_to_delete)].reset_index(drop=True)


In [16]:
name_to_delete = 'ronaldo@Student_21WU0102047'
retrive_df = retrive_df[~retrive_df['name_role'].str.contains(name_to_delete)].reset_index(drop=True)

In [17]:
retrive_df

,name_role,facial_features
0,Abhijeeth@Student,"[-0.02814327, -0.0032706298, -0.039785348, -0...."
1,kohli@Student,"[-1.1096125, 0.37213457, 0.12171401, -0.682275..."
2,abhigna@Student,"[-2.0028722, -0.5104687, 0.71650857, -0.830376..."
3,Lionel Messi@Student,"[-2.7039468, -1.4024091, 0.39546552, 0.6817208..."
4,Morgan_Freeman@teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711..."
5,Barack_Obama@teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ..."
6,Nihitha Vadlamuri@Student,"[0.40539905, -0.93340147, 0.6084892, -0.948747..."
7,Scarlett Johansson@student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578..."
8,Chris_Evans@student,"[0.72012275, -1.4683539, -0.77757514, -1.46026..."
9,Angelina Jolie@student,"[0.43222713, 1.074822, -0.12857957, -1.8332443..."


In [18]:
retrive_df


,name_role,facial_features
0,Abhijeeth@Student,"[-0.02814327, -0.0032706298, -0.039785348, -0...."
1,kohli@Student,"[-1.1096125, 0.37213457, 0.12171401, -0.682275..."
2,abhigna@Student,"[-2.0028722, -0.5104687, 0.71650857, -0.830376..."
3,Lionel Messi@Student,"[-2.7039468, -1.4024091, 0.39546552, 0.6817208..."
4,Morgan_Freeman@teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711..."
5,Barack_Obama@teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ..."
6,Nihitha Vadlamuri@Student,"[0.40539905, -0.93340147, 0.6084892, -0.948747..."
7,Scarlett Johansson@student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578..."
8,Chris_Evans@student,"[0.72012275, -1.4683539, -0.77757514, -1.46026..."
9,Angelina Jolie@student,"[0.43222713, 1.074822, -0.12857957, -1.8332443..."


### Registration Form

In [19]:
import cv2

from insightface.app import FaceAnalysis

In [20]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
# warning: don't set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


##### Step-1: Collect Person and Role

In [ ]:
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """)
    
    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        
        break
    else:
        print('Invalid entry. Try again.')
        if i == 2:
            print('Exceeds maximum trials')
            exit()

key = person_name + '@' + role
print('Your name = ', person_name)
print('Your role =', role)

print('Key =', key)


##### Step-2: Collect Samples

In [ ]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)
        
        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
        
    if sample >= 200: 
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this is trigger only when I press letter q (lowercase q) in my keyboard
        break
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


In [ ]:
len(face_embeddings)

##### Step-3: Optimize Data & Save in Redis Database

In [ ]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

In [ ]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [ ]:
# save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)